Target fits file will have its background and relative signal strength matched with reference frame. 
Both fits files MUST have the same pixel scale, rotation, astrometric solution, etc.
Resample FITS files together BEFORE using the linear fit function!

* This is basically just a python implementation of the LinearFit process in PixInsight, I got the algorithm for how to do this process from: https://pixinsight.com/forum/index.php?threads/linear-fit-rgb.7142/
Written by Jackson Datkuliak

In [1]:
# Imports
from astropy.io import fits
import numpy as np
from sklearn.linear_model import LinearRegression
import time

In [2]:
# Vars (LOOK HERE BEFORE RUNNING)
reference_path = "/home/jackson/test_fits/h.fit"   # Reference image read path
target_path = "/home/jackson/test_fits/o.fit"      # Target image read path
fitted_path = "/home/jackson/test_fits/fitted.fit" # Fitted image write path (Existing file will be overwritten!)
hdul_index = 0                                     # Change what HDUL index to grab image data from (usually 0)              Default = 0
verbose = True                                     # Report coefficient, intercept, and score of the linear regression model Default = True
debug_mode = False                                 # Debug mode. Only compute linear regression model                         Default = False

In [3]:
# start timer
if (verbose): 
    start = time.time()

try:
    # Load reference FITS file
    with fits.open(reference_path) as hdul_ref:
        ref_data = hdul_ref[hdul_index].data # grab reference frame image
    hdul_ref.close # close HDUL
except:
    raise Exception("Unable to open reference fits file.")
try:
    # Load target FITS file
    with fits.open(target_path) as hdul_tar:
        tar_data = hdul_tar[hdul_index].data # grab target frame image
        fitted = tar_data # copy target frame into new array to mutate later
        hdul_fitted = hdul_tar.copy() # copy HDUL to transfer to new fitted image
    hdul_tar.close # close old HDUL unit
except:
    raise Exception("Unable to open reference fits file.")

# Ensure fits files have the same dimensions
if (ref_data.size != tar_data.size):
    raise Exception("Reference and target FITS files were NOT the same size! Please resample images before performing linear fit!")

# Report that program found valid reference and target images
if (verbose): 
    print("Successfully loaded reference and target images.")

try:
    # Compute linear fitting function
    model = LinearRegression() # create linear regression model
    ref_data_res = ref_data.reshape(-1,1) # reshaping the data so the model will return one coef and intercept
    tar_data_res = tar_data.reshape(-1,1) # ...
    model.fit(tar_data_res, ref_data_res) # fitting model to target pixel value vs reference target pixel
    coef = model.coef_[0][0] # convert the coef and intercept from a numpy array to a float
    intercept = model.intercept_[0] # ...
    score = model.score(ref_data_res, tar_data_res) # get model fit score (doesn't necessarily need to be a good fit to work properly)
except:
    raise Exception("Unable to compute linear regression model.")

if (verbose): # report fit statistics
    print("coef : ", coef, "intercept: ", intercept, "score: ", score)

if (not debug_mode):
    try:
        # Apply linear fit function to new image array
        with np.nditer(fitted, op_flags=["readwrite"]) as x:
            for y in x:
                y[...]*=coef # multiply every pixel by coefficient
                y[...]+=intercept # add intercept to every pixel
    except:
        raise Exception("Unable to apply regression model to target image.")
    try:
        hdul_fitted.pop(hdul_index) # remove old image data
        hdul_fitted.insert(hdul_index, fits.PrimaryHDU(data=fitted)) # add fitted image data
        hdul_fitted.writeto("/home/jackson/test_fits/fitted.fit", overwrite=True) # write new fits image file
        if (verbose):
            print("Fitted FITS file written successfully.")
    except:
        raise Exception("Unable to write fitted fits file.")
    
if (verbose): # report time it took to compute
    end = time.time()
    print("Linear fit completed in", round(end - start, 2), "seconds.")


Exception: Reference and target FITS files were NOT the same size! Please resample images before performing linear fit!